# Week 4 - Multiclass Logistic Regression and Neural Networks

In this exercise, you will implement one-vs-all logistic regression and neural
networks to recognize hand-written digits.

## Multiclass Classification

We'll first tackle handwritten digit classification via logistic regression, implementing the one-vs-all classification technique for multiple classes. Each digit represents each digit, 0-9.

### Loading the Data
The training data is available as a matlab data file, so we have to read them in approprately. The training data contains 5000 samples, where each sample is a 20 by 20 pixel grayscale image of a single digit. Each pixel value is represented by a floating point number indicating the intensity of the pixel. As a result, our training data input matrix will be

$$ X \in \mathbb{R}^{5000 \times 400} $$

and our output vector,

$$ y \in \mathbb{R}^{5000} $$

will hold the actual values of each training sample. Now let's load the data.

In [1]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import optimize
import warnings
warnings.filterwarnings('ignore')

# Set the data path
dataDir = "./assignment/ex3/"
dataFile = "ex3data1.mat"
dataPath = dataDir+dataFile

# Load the data
data = loadmat(dataPath)
X = data['X']
y = data['y']

### Functions for Logistic Regression

We'll use the one-vs-all logistic regression models, so we need to train 10 separate logistic regression classification. Since this requires a lot more computation that our previous assignments, we need to be sure our code is vectorized. Fortunately, I thought that was the goal all along and I've been vectorizing my code anyways, so we can utilize some of the code from the previous assignment. We'll create some functions here to evaluate the sigmoid function, calculate the cost and gradients, and to perform the BFGS minimization of the cost function.

#### Sigmoid Function

First, we need to create the sigmoid function that will calculate

$$ g(z) = \frac{1}{1 + \exp(-z)} $$

In [2]:
def sigmoid(z):
    
    """Calculate the value of the sigmoid function for a given input"""
    
    # Return the sigmoid function
    return (1.0 / (1.0 + np.exp(-z)))

#### Cost and Gradient, Regularized

Next, we need to create functions to calculate the cost function and gradients to be used in our minimization. With regularization, our cost function is 

$$
J (\theta) = \frac{1}{2 m} \sum_{i=1}^m \Big( h_\theta (x^{(i)} - y^{(i)} \Big)^2 + \lambda \sum_{j=1}^n \theta_j^2
$$

And the regularized gradient is 

$$
\begin{align}
    \theta_0 & := \theta_0 - \alpha \frac{1}{m} \sum_{i=1}^m \Big( h_\theta (x^{(i)}) - y^{(i)} \Big) x_0^{(i)} \\
    \theta_j & := \theta_j - \alpha \frac{1}{m} \sum_{i=1}^m \Big( h_\theta (x^{(i)}) - y^{(i)} \Big) x_j^{(i)} + \frac{\lambda}{m} \theta_j,  \text{ for } j \in \{1,2,\cdots,n\} \\
    & := \theta_j(1-\alpha\frac{\lambda}{m}) - \alpha\frac{1}{m} \sum_{i=1}^m \Big( h_\theta (x^{(i)}) - y^{(i)} \Big) x_j^{(i)}
\end{align}
$$

Recall that the minimization function will calculate the learning rate for us.

In [3]:
def regularized_cost(theta,X,y,regCoeff):
    
    """Calculate the regularized cost function for logistic regression"""
    
    # Specify arrays as matrices
    X = np.matrix(X)
    y = np.matrix(y).T
    theta = np.matrix(theta).T
    
    # Get number of training samples
    m = len(y)
    
    # Calculate the hypothesis function
    h = sigmoid(X*theta)
    
    # Calculate the unregularized cost function
    J = (1.0/m)*(-y.T*np.log(h) - (1.0-y).T*np.log(1.0-h))
    
    # Returen the regularized cost function 
    return (J + (regCoeff/(2.0*m))*(theta[1:].T*theta[1:])).item()

def regularized_gradient(theta,X,y,regCoeff):
    
    """Calculate the regularized gradient terms of the cost function"""
   
    # Specify arrays as matrices
    X = np.matrix(X)
    y = np.matrix(y).T
    theta = np.matrix(theta).T

    # Get number of training samples
    m = len(y)
    
    # Calculate the hypothesis function
    h = sigmoid(X*theta)
    
    # Calculate the unregularized gradient
    grad = (1.0/m)*(X.T*(h-y));
    
    # Return the regularize gradient
    grad[1:] = grad[1:]+((regCoeff/m)*(theta[1:]));
    return grad

#### Minimization and One-vs-All

Now we create a function to minimize the weights for the hypothesis and then another function to run this minimization on all of our classes. The results of this will be all of the weights for each of the classes. We can then use this to compute the prediction on a given input. 

I've seen a lot of people use the "TNC", or Truncated Newton Conjugate-Gradient method for minimizing a function, so I will use that as well. Note that the internal scipy code for each method is written differently, so if we want to change our method, we'll need to change some of our arrays to match the proper dimensions needed for that method's computation.

In [4]:
def minimize(cost,gradient,theta,args):

    """Use the scipy optimize BFGS minimization function to minimize the cost function"""
    
    # Set up the arguments
    fun = cost
    x0 = theta
    fprime = gradient
    args = args

    # Return the minimized parameters
    return optimize.minimize(fun=fun,x0=theta,args=args,method='TNC',jac=gradient)

def one_vs_all(X,y,nLabels,regCoeff):

    '''Run a minimization function on each class for a given training set'''
    
    # Set array size
    m,n = X.shape

    # Set array for the parameters of each of the classifiers
    allTheta = np.zeros((nLabels,n+1))

    # Insert a column of ones at the beginning for the intercept term
    X = np.concatenate((np.ones((m,1)),X),axis=1)
    y = y.ravel()

    # Loop through labels 1-10
    for i in range(1, nLabels+1):

        # Set arrays for the minimization function
        theta = np.zeros(n+1)
        y_i = np.array([1 if label == i else 0 for label in y])

        # Minimize the objective function
        fmin = minimize(regularized_cost,regularized_gradient,theta,(X,y_i,regCoef))
        allTheta[i-1,:] = fmin.x
        
    return allTheta

#### Predict Results

To get the results of our classification, we need to calculate

$$ \Theta = \texttt{allTheta} \\
   h( X \Theta^T ) = \frac{1}{1 + \exp(-X \Theta^T)} $$
   
where our hypothesis gives a probability of the input beloging to each class. We then want to get the class for each input where the probability of that input is maximized.

In [5]:
def predict(allTheta,X):

    # Get X sizes
    m,n = X.shape
    
    # Insert column of ones 
    X = np.insert(X, 0, values=np.ones(m), axis=1)
    
    # Convert to matrices
    X = np.matrix(X)
    allTheta = np.matrix(allTheta)
    
    # Compute the probability for each class on each training instance
    h = sigmoid(X * allTheta.T)
    
    # Create array of the index with the maximum probability
    hMax = np.argmax(h, axis=1)
    
    # Since our array was zero-indexed we need to add one for the true label prediction
    hMax = hMax + 1
    
    return hMax

### Implementation

Now let's implement the functions we created above.

In [6]:
# Reload X and y
X = data['X']
y = data['y']

# Set learning and classification parameters
regCoef = 0.1
nLabels = 10

# Get the weights from the minimization on the 10 layers
allTheta = one_vs_all(X,y,nLabels,regCoef)

#### Predicting the Classes

Let's now evaluate the highest probability classes for the training set and evaluate the accuracy.

In [7]:
# Run the prediction on the training set
yPredicted = predict(allTheta,X)

# Calculate the accuracy of the prediction to the true training samples
correct = [1 if a == b else 0 for (a, b) in zip(yPredicted, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('Training accuracy = {0:0.2f}%'.format(accuracy * 100))

Training accuracy = 96.46%


## Neural Networks

In this part of the assignment, we'll use a neural network as our model. Our NN is shown below:

![Our NN for Assignment 4](./assignment_nn.png "Our NN for Assignment 4")

We are provided with the weights for the NN. We'll learn how to produce those weights next week.

### Loading the data

Let's first load the data and the weights

In [8]:
# Set the data path
dataDir = "./assignment/ex3/"
dataFile = "ex3data1.mat"
dataPath = dataDir+dataFile

# Set the weights path
weightsFile = "ex3weights.mat"
weightsPath = dataDir+weightsFile

# Load the data
data = loadmat(dataPath)
X = data['X']
y = data['y']

# Load the weights
weights = loadmat(weightsPath)
Theta1 = weights['Theta1']
Theta2 = weights['Theta2']

### Predicting from the Neural Network Weights

Since we already have the weights, using the NN as a model is actually pretty simple.  We will first set

$$ a_1 = [ 1_{m \times 1} | X ] $$

then just implement the following routine to get to the output, $a3$:

$$
\begin{align}
z_2 & = a_1 \Theta_1^T \\
a_2 & = [ 1_{s_2 \times 1} | g (z_2) ] \\
z_3 & = a_2 \Theta_2^T \\
a_3 & = g (z_3)
\end{align}
$$

where $s_i$ is the number of rows in $z_i$.

### Implementation and Prediction

Let's implement this now.

In [9]:
# Construct a1
m = X.shape[0]
a1 = np.concatenate((np.ones((m,1)),X),axis=1)

# Calculate z2
a1 = np.matrix(a1)
Theta1 = np.matrix(Theta1)
z2 = a1*Theta1.T

# Calculate a3
m = z2.shape[0]
a2 = np.concatenate((np.ones((m,1)),sigmoid(z2)),axis=1)

# Calculate z3
a2 = np.matrix(a2)
Theta2 = np.matrix(Theta2)
z3 = a2*Theta2.T

# Calculate a3
a3 = sigmoid(z3)

# Create array of the index with the maximum probability
yPredicted = np.argmax(a3, axis=1)

# Since our array was zero-indexed we need to add one for the true label prediction
yPredicted = yPredicted + 1

# Calculate the accuracy of the prediction to the true training samples
correct = [1 if a == b else 0 for (a, b) in zip(yPredicted, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('Training accuracy = {0:0.2f}%'.format(accuracy * 100))

Training accuracy = 97.52%
